In [ ]:
import glob
import numpy as np
import scipy.io as sio
GOOGLE_COLAB = True
path = ""
if GOOGLE_COLAB:
    from google.colab import drive, files
    drive.mount('/content/drive/')

EEG_data_directory_path = '/content/drive/MyDrive/ML/project/EEG_Data/25-users/*.txt'
Labels_directory_path = '/content/drive/MyDrive/ML/project/EEG_Data/labels/*.lab'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# For the EEG Data

data_file_paths = glob.glob(EEG_data_directory_path)

flattened_data = np.empty((0, 7168), dtype=float)

for data_file_path in data_file_paths:

    data = np.loadtxt(data_file_path)


    flattened_row = data.flatten().reshape(1, -1)


    flattened_data = np.concatenate((flattened_data, flattened_row), axis=0)

print("Shape of Flattened EEG Data:", flattened_data.shape)

Shape of Flattened EEG Data: (1045, 7168)


In [ ]:
# For Labels
label_file_paths = glob.glob(Labels_directory_path)

labels = []

label_mapping = {"disike": 0, "like": 1}

for label_file_path in label_file_paths:
    with open(label_file_path, 'r') as label_file:
        label_word = label_file.read().strip().lower()
        label_value = label_mapping.get(label_word, None)
        if label_value is not None:
            labels.append(label_value)

labels = np.array(labels).reshape(-1, 1)

print("Shape of Labels:", labels.shape)

Shape of Labels: (1045, 1)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Combining the Flattened EEG Data with respective Labels
combined_data = np.hstack((flattened_data, labels))

print("Shape of Combined Data:", combined_data.shape)

# Extract features and labels from the combined data
eeg_features, labels = combined_data[:, :-1], combined_data[:, -1]

# Standardize the features (mean=0, variance=1)
scaler = StandardScaler()
eeg_features_standardized = scaler.fit_transform(eeg_features)

# Apply Principal Component Analysis (PCA) for dimensionality reduction
# Adjust n_components based on your requirements
n_components = 50
pca = PCA(n_components=n_components)
eeg_features_pca = pca.fit_transform(eeg_features_standardized)

# Combine the preprocessed features with labels
preprocessed_data = np.hstack((eeg_features_pca, labels.reshape(-1, 1)))

print("Shape of Preprocessed Data:", preprocessed_data.shape)

Shape of Combined Data: (1045, 7169)
Shape of Preprocessed Data: (1045, 51)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


X_train, X_test, y_train, y_test = train_test_split(preprocessed_data[:, :-1], preprocessed_data[:, -1], test_size=0.2, random_state=42)

# Initialize the logistic regression model
logreg_model = LogisticRegression()

# Train the model on the training data
logreg_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg_model.predict(X_test)

# Calculate and print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the logistic regression model:", accuracy)

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy of the logistic regression model: 0.7703349282296651
Classification Report:
              precision    recall  f1-score   support

         0.0       0.77      0.83      0.80       114
         1.0       0.78      0.69      0.73        95

    accuracy                           0.77       209
   macro avg       0.77      0.76      0.77       209
weighted avg       0.77      0.77      0.77       209



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'preprocessed_data' is your preprocessed and feature-extracted data

# Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data[:, :-1], preprocessed_data[:, -1], test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model = SVC()

# Train the model on the training data
svm_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm_model.predict(X_test)

# Calculate and print the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the SVM model:", accuracy)

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy of the SVM model: 0.7942583732057417
Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.78      0.81       114
         1.0       0.75      0.81      0.78        95

    accuracy                           0.79       209
   macro avg       0.79      0.80      0.79       209
weighted avg       0.80      0.79      0.79       209



In [ ]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import layers

# Assuming 'preprocessed_data' is your preprocessed and feature-extracted data

# Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data[:, :-1], preprocessed_data[:, -1], test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the ANN model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Make predictions on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Epoch 1/10
21/21 [==============================] - 1s 7ms/step - loss: 0.6434 - accuracy: 0.6632 - val_loss: 0.5973 - val_accuracy: 0.7024
Epoch 2/10
21/21 [==============================] - 0s 2ms/step - loss: 0.5268 - accuracy: 0.7590 - val_loss: 0.5451 - val_accuracy: 0.7202
Epoch 3/10
21/21 [==============================] - 0s 2ms/step - loss: 0.4471 - accuracy: 0.7889 - val_loss: 0.5023 - val_accuracy: 0.7381
Epoch 4/10
21/21 [==============================] - 0s 3ms/step - loss: 0.3871 - accuracy: 0.8099 - val_loss: 0.4723 - val_accuracy: 0.7262
Epoch 5/10
21/21 [==============================] - 0s 3ms/step - loss: 0.3416 - accuracy: 0.8383 - val_loss: 0.4518 - val_accuracy: 0.7560
Epoch 6/10
21/21 [==============================] - 0s 2ms/step - loss: 0.3045 - accuracy: 0.8653 - val_loss: 0.4326 - val_accuracy: 0.7560
Epoch 7/10
21/21 [==============================] - 0s 2ms/step - loss: 0.2695 - accuracy: 0.8922 - val_loss: 0.4140 - val_accuracy: 0.7738
Epoch 8/10
21/21 [==